[View in Colaboratory](https://colab.research.google.com/github/krapshsa/tensorflow-practice/blob/master/autoencoder.ipynb)

In [5]:
from __future__ import division, print_function, absolute_import

import tensorflow as tf
import numpy as np

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


# Training Parameters
batch_size=256
max_steps=1000
display_steps=1000
learning_rate=0.01


# Network Parameters
num_hidden_1 = 256 # 1st layer num features
num_hidden_2 = 128 # 2nd layer num features (the latent dim)

time_steps = 28
feature_size = 28

tf.reset_default_graph()

# tf Graph input (only pictures)
'''
[
    [
      [feature_1, feature_2, ... feature_n] # timestep_1
      [feature_1, feature_2, ... feature_n] # timestep_2
      ...
      [feature_1, feature_2, ... feature_n] # timestep_n
    ], #batch_1
    ...
]
shape=(batches, steps, features)
'''
X = tf.placeholder("float", [None, time_steps, feature_size])

def autoencoder(x):
  # Construct model
  rnn_layers = [
      tf.nn.rnn_cell.LSTMCell(
        num_units=units,
        forget_bias=1.0,
        activation=tf.nn.tanh
      ) for units in [num_hidden_1, num_hidden_2, num_hidden_1, feature_size]
  ]

  # create a RNN cell composed sequentially of a number of RNNCells
  multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
  '''
  [
      [
        [feature_1, feature_2, ... feature_n] # (batch_1 , timestep_1)
        [feature_1, feature_2, ... feature_n] # (batch_2 , timestep_1)
        ...
        [feature_1, feature_2, ... feature_n] # (batch_n , timestep_1)
      ],
      ...
  ]
  shape=(steps, batches, features)
  '''
  inputs = tf.unstack(
      value=x,
      num=time_steps,
      axis=1
  )
  outputs, states = tf.nn.static_rnn(
      cell=multi_rnn_cell,
      inputs=inputs,
      dtype=tf.float32
  )
  
  
  '''
  [
      [
        [feature_1, feature_2, ... feature_n] # (batch_1 , timestep_1)
        [feature_1, feature_2, ... feature_n] # (batch_2 , timestep_1)
        ...
        [feature_1, feature_2, ... feature_n] # (batch_n , timestep_1)
      ],
      ...
  ]
  
  shape=(steps, batches, features)
  '''
  return outputs

# slice to keep only the last cell of the RNN

outputs = autoencoder(X)
'''
[
    [
      [feature_1, feature_2, ... feature_n] # timestep_1
      [feature_1, feature_2, ... feature_n] # timestep_2
      ...
      [feature_1, feature_2, ... feature_n] # timestep_n
    ], #batch_1
    ...
]
shape=(batches, steps, features)
'''
stack_outputs = tf.stack(
    values=outputs,
    axis=1
)

# Prediction
y_pred = stack_outputs
# Targets (Labels) are the input data.
y_true = X

print(tf.convert_to_tensor(outputs).get_shape())
print(y_pred.get_shape())
print(y_true.get_shape())

# Define loss and optimizer, minimize the squared error
loss = tf.reduce_mean(tf.square(tf.subtract(y_true, y_pred)))
optimizer = tf.train.RMSPropOptimizer(learning_rate)
train_op = optimizer.minimize(loss)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start Training
# Start a new TF session
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Training
    for i in range(1, max_steps+1):
        # Prepare Data
        # Get the next batch of MNIST data (only images are needed, not labels)
        batch_x, _ = mnist.train.next_batch(batch_size)
        sequence_x = batch_x.reshape((batch_size, time_steps, feature_size))

        # Run optimization op (backprop) and cost op (to get loss value)
        sess.run(train_op, feed_dict={X: sequence_x})

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
(28, ?, 28)
(?, 28, 28)
(?, 28, 28)
